## Human action recognition ##
By Mark Putter - s3945532

In [ ]:
import tensorflow as tf
import datetime, os

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# load data
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Create model
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

# Compile and train
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


: 

In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [ ]:
model.fit(x=x_train,
            y=y_train,
            epochs=50,
            verbose=0,
            validation_data=(x_test, y_test),
            callbacks=[tensorboard_callback])

In [ ]:
import tensorflow_datasets as tfds

dataset, info = tfds.load('fashion_mnist', with_info=True, as_supervised=True)
fmnist_train, fmnist_test = dataset['train'], dataset['test']

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(16,4))
i=1
for image, label in fmnist_train.take(4):  # example is (image, label)
  plt.subplot(1,4,i)
  plt.imshow(np.squeeze(image))
  i = i+1

plt.show()

In [ ]:
# Function to Preprocess the dataset
def convert_types(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255
  return image, label

batch_size = 32
fmnist_train = fmnist_train.map(convert_types).shuffle(10000).batch(batch_size)
fmnist_test = fmnist_test.map(convert_types).batch(batch_size)

In [ ]:
for image, label in fmnist_train.take(3):  # example is (image, label)
  print(image.shape, label)

In [ ]:
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Input
from tensorflow import keras
from tensorflow.keras import Model


class FMNISTModel(Model):
  def __init__(self):
      super(FMNISTModel, self).__init__()
      self.conv1 = Conv2D(64, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01), input_shape=(28, 28, 1))
      self.pool1 = MaxPooling2D((2, 2))
      self.flatten = Flatten()
      self.d1 = Dense(128, activation='relu')
      self.drop = Dropout(0.2)
      self.d2 = Dense(10, activation='softmax')

  def call(self, x):
      x = self.conv1(x)
      x = self.pool1(x)
      x = self.flatten(x)
      x = self.d1(x)
      x = self.drop(x)
      return self.d2(x)

model = FMNISTModel()

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [ ]:
@tf.function
def train_step(image, label):
  with tf.GradientTape() as tape:
    predictions = model(image)
    loss = loss_object(label, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(label, predictions)

In [ ]:
@tf.function
def test_step(image, label):
  predictions = model(image)
  t_loss = loss_object(label, predictions)

  test_loss(t_loss)
  test_accuracy(label, predictions)

In [ ]:
EPOCHS = 5

for epoch in range(EPOCHS):
  print("starting Epoch: ", epoch)
  # iterate over all batches in the dataset
  for image, label in fmnist_train:
    train_step(image, label)

  for test_image, test_label in fmnist_test:
    test_step(test_image, test_label)

  print('Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))

  train_loss.reset_state()
  test_loss.reset_state()
  train_accuracy.reset_state()
  test_accuracy.reset_state()